# Three investigators - part I.I

A project for scraping and analysing data from a fan site on the audio book called '[The three investigators](https://en.wikipedia.org/wiki/Three_Investigators#Germany)'

Part I.I: Topic modelling

Using the content and title for each episode to detect the overall topic.

In [1]:
#python version used for this project
from platform import python_version
print(python_version())

3.7.5


> need to add additional modules to requirement text file !!

In [2]:
# import modules [as specified in requirements.txt]
import pandas as pd
import numpy as np
import spacy
import nltk
import re

# for file 
import os

%matplotlib inline

In [3]:
#change directory to root folder
os.chdir("..")

## Load data

In [51]:
#load scraped datafiles
meta = pd.read_csv(".\\data\\scraped\\meta.csv")
content = pd.read_csv(".\\data\\scraped\\content_all.csv")

# make all column names lower case
df_list = [meta, content]

for df in df_list:
    df.columns = df.columns.str.lower()

## Title

### Standardising

In [85]:
## make titles lower case
title = meta["titel"].str.lower()

# replace values within titles

# function to loop through the column and replace substrings
def replace_values(text, dic):
    for x, y in dic.items():
        text = text.str.replace(x, y, regex=True)
    return text

# list of values to be replaced, including punctuation
replace_dict = {"hörspiel": "", 
                "[!\"#$%&()*+,./:;<=>?@[\]^_`{|}~“”-]": ""}

# apply function
title = replace_values(title, replace_dict)

# strip white space at the end
title = title.str.strip()

title

0             der superpapagei
1               der phantomsee
2             der karpatenhund
3           die schwarze katze
4         der fluch des rubins
                ...           
200             das weiße grab
201    tauchgang ins ungewisse
202         der dunkle wächter
203       das rätselhafte erbe
204         und der mottenmann
Name: titel, Length: 205, dtype: object

### remove stopwords

In [77]:
# download German stop words
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('german'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lisa.hornung\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [86]:
# split titles into substrings using space as delimiter
title_split = title.str.split(" ")

# create empty list to store titles without stop words
title_no_stop_words = []

# iterate through each word in each title and append those that are no stop words
for words in title_split:
    x = []
    for word in words:
        if word not in stop_words:
            x.append(word)
    title_no_stop_words.append(x)

title_no_stop_words = [" ".join(title_no_stop_words[i]) for i in range(len(title_no_stop_words))]
title_no_stop_words

['superpapagei',
 'phantomsee',
 'karpatenhund',
 'schwarze katze',
 'fluch rubins',
 'sprechende totenkopf',
 'unheimliche drache',
 'grüne geist',
 'rätselhaften bilder',
 'flüsternde mumie',
 'gespensterschloß',
 'seltsame wecker',
 'lachende schatten',
 'bergmonster',
 'rasende löwe',
 'zauberspiegel',
 'gefährliche erbschaft',
 'geisterinsel',
 'teufelsberg',
 'flammende spur',
 'tanzende teufel',
 'verschwundene schatz',
 'aztekenschwert',
 'silberne spinne',
 'singende schlange',
 'silbermine',
 'magische kreis',
 'doppelgänger',
 'riff haie',
 'narbengesicht',
 'ameisenmensch',
 'bedrohte ranch',
 'rote pirat',
 'höhlenmensch',
 'superwal',
 'heimliche hehler',
 'unsichtbare gegner',
 'perlenvögel',
 'automarder',
 'volk winde',
 'weinende sarg',
 'höllische werwolf',
 'gestohlene preis',
 'gold wikinger',
 'schrullige millionär',
 'giftige gockel',
 'gefährlichen fässer',
 'comicdiebe',
 'verschwundene filmstar',
 'riskante ritt',
 'musikpiraten',
 'automafia',
 'gefahr verzug

# Content

## Standardising

In [88]:
# list of acronyms to be replaced
replace_dict = {"" : ""}

## make content lower case
title = content["content"].str.lower()

# replace values within titles

# function to loop through the column and replace substrings
def replace_values(text, dic):
    for x, y in dic.items():
        text = text.str.replace(x, y, regex=True)
    return text

# list of values to be replaced, including punctuation
replace_dict = {"[!\"#$%&()*+,./:;<=>?@[\]^_`{|}~“”-]": ""}


# apply function
title = replace_values(title, replace_dict)

# strip white space at the end
title = title.str.strip()

title

0      Der neueste Auftrag an die drei Detektive hört...
1      Welches Geheimnis verbirgt sich in einem vergi...
2      "Bei mir spukt es!" Mit diesem verzweifelten A...
3      In einem kleinen Wanderzirkus wittern die drei...
4      Alfred Hitchcock und die drei Detektive (Firme...
                             ...                        
201    So hatten sich die drei Detektive ihre Auszeit...
202    Ein Kindermädchen, das nachts in Gestalt eines...
203    Eigentlich sollte Bob in dem einsam gelegenen ...
204    Ein merkwürdiger Anruf erreicht Die Drei Frage...
205    In Rocky Beach taucht eine schaurige Gestalt a...
Name: content, Length: 206, dtype: object